In [21]:
import time
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import pandas as pd
import numpy as np
from model import SSD300, MultiBoxLoss
from datasets import PascalVOCDataset
from collections import defaultdict
from utils import *
from pathlib import Path
from random import shuffle



# Data parameters
data_folder = './'  # folder with data files
keep_difficult = True  # use objects considered difficult to detect?
# Model parameters
# Not too many here since the SSD300 has a very specific structure
n_classes = len(label_map)  # number of different types of objects
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Learning parameters
checkpoint = None  # path to model checkpoint, None if none
batch_size = 16  # batch size
start_epoch = 0  # start at this epoch
epochs = 200  # number of epochs to run without early-stopping
epochs_since_improvement = 0  # number of epochs since there was an improvement in the validation metric
best_loss = 100.  # assume a high loss at first
workers = 4  # number of workers for loading data in the DataLoader
print_freq = 200  # print training or validation status every __ batches
lr = 1e-3  # learning rate
momentum = 0.9  # momentum
weight_decay = 5e-4  # weight decay
grad_clip = None  # clip if gradients are exploding, which may happen at larger batch sizes (sometimes at 32) - you will recognize it by a sorting error in the MuliBox loss calculation

cudnn.benchmark = True


In [20]:
from torchvision import transforms
from utils import *
from PIL import Image, ImageDraw, ImageFont

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load model checkpoint
checkpoint = 'BEST_checkpoint_ssd300.pth.tar'
checkpoint = torch.load(checkpoint)
start_epoch = checkpoint['epoch'] + 1
best_loss = checkpoint['best_loss']
print('\nLoaded checkpoint from epoch %d. Best loss so far is %.3f.\n' % (start_epoch, best_loss))
model = checkpoint['model']
model = model.to(device)
model.eval()

# Transforms
resize = transforms.Resize((300, 300))
to_tensor = transforms.ToTensor()
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])


def detect(original_image, min_score, max_overlap, top_k, suppress=None):
    """
    Detect objects in an image with a trained SSD300, and visualize the results.

    :param original_image: image, a PIL Image
    :param min_score: minimum threshold for a detected box to be considered a match for a certain class
    :param max_overlap: maximum overlap two boxes can have so that the one with the lower score is not suppressed via Non-Maximum Suppression (NMS)
    :param top_k: if there are a lot of resulting detection across all classes, keep only the top 'k'
    :param suppress: classes that you know for sure cannot be in the image or you do not want in the image, a list
    :return: annotated image, a PIL Image
    """

    # Transform
    image = normalize(to_tensor(resize(original_image)))

    # Move to default device
    image = image.to(device)

    # Forward prop.
    predicted_locs, predicted_scores = model(image.unsqueeze(0))

    # Detect objects in SSD output
    det_boxes, det_labels, det_scores = model.detect_objects(predicted_locs, predicted_scores, min_score=min_score,
                                                             max_overlap=max_overlap, top_k=top_k)

    # Move detections to the CPU
    det_boxes = det_boxes[0].to('cpu')

    # Transform to original image dimensions
    original_dims = torch.FloatTensor(
        [original_image.width, original_image.height, original_image.width, original_image.height]).unsqueeze(0)
    det_boxes = det_boxes * original_dims

    # Decode class integer labels
    det_labels = [rev_label_map[l] for l in det_labels[0].to('cpu').tolist()]

    # If no objects found, the detected labels will be set to ['0.'], i.e. ['background'] in SSD300.detect_objects() in model.py
    if det_labels == ['background']:
        # Just return original image
        return original_image

    # Annotate
    annotated_image = original_image
    draw = ImageDraw.Draw(annotated_image)
    font = ImageFont.truetype("./calibril.ttf", 15)

    # Suppress specific classes, if needed
    for i in range(det_boxes.size(0)):
        if suppress is not None:
            if det_labels[i] in suppress:
                continue

        # Boxes
        box_location = det_boxes[i].tolist()
        draw.rectangle(xy=box_location, outline=label_color_map[det_labels[i]])
        draw.rectangle(xy=[l + 1. for l in box_location], outline=label_color_map[
            det_labels[i]])  # a second rectangle at an offset of 1 pixel to increase line thickness
        # draw.rectangle(xy=[l + 2. for l in box_location], outline=label_color_map[
        #     det_labels[i]])  # a third rectangle at an offset of 1 pixel to increase line thickness
        # draw.rectangle(xy=[l + 3. for l in box_location], outline=label_color_map[
        #     det_labels[i]])  # a fourth rectangle at an offset of 1 pixel to increase line thickness

        # Text
        text_size = font.getsize(det_labels[i].upper())
        text_location = [box_location[0] + 2., box_location[1] - text_size[1]]
        textbox_location = [box_location[0], box_location[1] - text_size[1], box_location[0] + text_size[0] + 4.,
                            box_location[1]]
        draw.rectangle(xy=textbox_location, fill=label_color_map[det_labels[i]])
        draw.text(xy=text_location, text=det_labels[i].upper(), fill='white')
    del draw

    return annotated_image

from pathlib import Path
if __name__ == '__main__':
    for img_path in list(Path('../TestingPicturesOnModel').glob('*.jpg')):
        print(img_path)
        original_image = Image.open(img_path, mode='r')
        original_image = original_image.convert('RGB')
        detect(original_image, min_score=0.2, max_overlap=0.5, top_k=200).show()
        break;



Loaded checkpoint from epoch 84. Best loss so far is 0.259.

../TestingPicturesOnModel/IMG_20190220_125237.jpg


In [22]:
train_dataset = PascalVOCDataset(data_folder,
                                     split='train',
                                     keep_difficult=keep_difficult)
val_dataset = PascalVOCDataset(data_folder,
                               split='test',
                               keep_difficult=keep_difficult)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                                           collate_fn=train_dataset.collate_fn, num_workers=workers,
                                           pin_memory=True)  # note that we're passing the collate function here
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True,
                                         collate_fn=val_dataset.collate_fn, num_workers=workers,
                                         pin_memory=True)

In [23]:
asdf = iter(train_dataset)

In [24]:
next(asdf)

RuntimeError: bool value of Tensor with more than one value is ambiguous

In [34]:
data = pd.read_csv('../pklot_non_rotated.csv')

In [35]:
data['image_path'] = data['image_path'].map(lambda x: '../' + x)
data['rotated_bbox'] = data['rotated_bbox'].map(lambda x: np.array([int(num) for num in x.split()]))
data['rotated_bbox'] = data['rotated_bbox'].map(lambda x: np.split(x, len(x) / 4))
data['occupied'] = data['occupied'].map(lambda x: [int(num) for num in x.split()])
data['image_path'] = data['image_path'].map(lambda x: Path(x))
data.head()

,image_path,occupied,rotated_bbox
0,../PKLot/PKLot/UFPR04/Rainy/2013-01-18/2013-01...,"[0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[[577, 554, 724, 717], [629, 486, 780, 639], [..."
1,../PKLot/PKLot/UFPR04/Rainy/2013-01-18/2013-01...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[[577, 554, 724, 717], [629, 486, 780, 639], [..."
2,../PKLot/PKLot/UFPR04/Rainy/2013-01-18/2013-01...,"[1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[[577, 554, 724, 717], [629, 486, 780, 639], [..."
3,../PKLot/PKLot/UFPR04/Rainy/2013-01-18/2013-01...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[[577, 554, 724, 717], [629, 486, 780, 639], [..."
4,../PKLot/PKLot/UFPR04/Rainy/2013-01-18/2013-01...,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[[577, 554, 724, 717], [629, 486, 780, 639], [..."


In [36]:
pics = Path().glob("../PKLot/PKLot/**/*.jpg")
path_to_cnt = defaultdict(int)
total = 0
for pic in pics:
    path_to_cnt[pic.parent] += 1
    total +=1
    
training = []
validation = []
cnt = 0
threshold = .8

dates = list(path_to_cnt.keys())
shuffle(dates)

for pic in dates:
    if (cnt < threshold * total):
        training.append(pic)
        cnt += path_to_cnt[pic]
    else:
        validation.append(pic)
        
training = set(training)
    
def pklot_train_test_split(image_path, train_test_set):
    return image_path.parent in train_test_set

In [37]:
train_df = data[data.apply(lambda x: pklot_train_test_split(x['image_path'], training), axis=1)]
test_df = data[data.apply(lambda x: pklot_train_test_split(x['image_path'], validation), axis=1)]